In [1]:
import torch 
import torch.nn as nn
import os
import time
import torchvision
import numpy as np
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import copy

plt.ion()

Here we will use the <code>torchvision.datasets.ImageFolder</code> class to see how to get data when data is structured in the following format.
<code>
    <b>root/class_name/image.jpg</b>
</code>

<p>We will also use <code><b>torchvision.transforms</b></code> to apply transformation to the data.</p>

In [2]:
transform = {
    'train': transforms.Compose([transforms.RandomResizedCrop(224),
                               transforms.ToTensor(),
                               transforms.RandomHorizontalFlip(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    
    'val': transforms.Compose([transforms.RandomResizedCrop(224),
                               transforms.ToTensor(),
                               transforms.RandomHorizontalFlip(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
}

directory = 'data/hymenoptera_data'
dataset = {x: datasets.ImageFolder(os.path.join(directory,x),transform=transform[x]) for x in ['train','val']} 

dataloader = {x:torch.utils.data.DataLoader(dataset[x], batch_size=4, shuffle=True, num_workers=2) 
              for x in ['train','val']
             }

dataset_sizes = {x:len(dataset[x]) for x in ['train','val']}

classes = dataset['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Training the model 

In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_weights = copy.deepcopy(model.state_dict)
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch : {}/{}'.format(epoch+1, num_epochs))
        print('-'*10)
        
        # Each dataset has a phase either train or validate
        for phase in ['train','val']:
            if phase == 'train':
                scheduler.step()
                model.train() # set model in training mode
            else:
                model.eval() # set model in evaluate mode
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data
            for inputs, labels in dataloader[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the gradients 
                optimizer.zero_grad()
                
                # track history/ gradients only if phase is train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _,preds = torch.max(outputs, 1)

                    # backward only if in training phase
                    if phase == 'train' :
                        loss.backward()
                        optimizer.step()

                # update the statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        
        print()
        
    # load best weights for model
    model.load_state_dict(best_model_wts)
    return model

In [4]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\athar/.cache\torch\checkpoints\resnet18-5c106cde.pth
100.0%


RuntimeError: CUDA error: unknown error